In [1]:
import json
import pickle
import csv
import pandas as pd

In [2]:
def pickleLoad(path):
    file = open(path, 'rb')
    info = pickle.load(file)
    file.close()
    return info

def pickleDump(var, destination):
    f = open(destination,"wb")
    pickle.dump(var,f)
    f.close() 

def getQA(d_path, group, bal_txt, v_id):
    path = '../../exports/%s/%s/%s/%s.txt' % (d_path, bal_txt, group, v_id)
    try:
        with open(path) as json_file: #CHANGED
            QA = json.load(json_file)
        return QA
    except:
        return {}

In [3]:
with open('../../data/templ2global.txt', 'r') as f:
    templ2global = json.load(f)

In [4]:
train_stsgs = pickleLoad('../../data/stsgs/train_stsgs.pkl')
test_stsgs = pickleLoad('../../data/stsgs/test_stsgs.pkl')

In [5]:
# Testing to CSV
#46GP8, N11GT, KRF68, MJO7C, S6MPZ

def toDataframe(v_id, QA):
    df = pd.DataFrame.from_dict(QA, orient="index")
    df['vid_id'] = v_id
    #df.index = v_id
    df['gif_name'] = v_id
    df = df.drop(columns=['attributes', 'global', 'id', 'local', 'steps', 'metrics', 'program', 'sg_grounding'])
    df = df.reset_index()
    df = df.rename(columns={'index': 'key'})
    df['description'] = "none"
    return df


def toTGIF(d_path, group, bal_txt):
    dataframes = []
    if group == 'train':
        our_data = train_stsgs
    if group == 'test':
        our_data = test_stsgs
        
    cnt = 0
    for v_id in our_data:
        if cnt % 1000 == 0:
            print(cnt)
        cnt = cnt + 1
        QA = getQA(d_path, group, bal_txt, v_id)
        
        # Add the following lines
        for q_id in QA:
            QA[q_id]['type'] = QA[q_id]['attributes']['type']
        if len(QA) == 0: 
            continue
        df = toDataframe(v_id, QA)
        dataframes.append(df)
    total = pd.concat(dataframes)
    total.to_csv('../../exports/%s/other_formats/%s-%s-tgif.csv' % (d_path, group, bal_txt), index = None)
    return total


# Build CSV files

In [53]:
d_path = 'dataset_separating for compo-steps'
bal_txt = 'balanced - main dataset'

In [54]:
train = toTGIF(d_path, 'train', bal_txt)

0
1000
2000
3000
4000
5000
6000
7000


In [88]:
test = toTGIF(d_path, 'test', bal_txt)

0
1000


In [89]:
# load the pickles
# drop other columns
# attach together
#training = pd.read_csv('../../exports/%s/other_formats/train-balanced-tgif.csv' % d_path)
#testing = pd.read_csv('../../exports/%s/other_formats/test-balanced-tgif.csv' % d_path)

In [90]:

total = pd.concat([train, test])
total

,key,question,answer,type,vid_id,gif_name,description
0,46GP8-5,"In the video, did they interact with a window?",Yes,objExists,46GP8,46GP8,none
1,46GP8-40,Were they interacting with a table?,No,objExists,46GP8,46GP8,none
2,46GP8-388,Which object did the person interact with afte...,window,objWhatGeneral,46GP8,46GP8,none
3,46GP8-389,"Before watching outside of a window, what was ...",food,objWhat,46GP8,46GP8,none
4,46GP8-390,"In the video, which object were they holding?",food,objWhat,46GP8,46GP8,none
...,...,...,...,...,...,...,...
56,LSKA2-2100,What did the person do before holding the thin...,taking food from somewhere,actWhatBefore,LSKA2,LSKA2,none
57,LSKA2-2101,"Before holding the object they were taking, wh...",taking food from somewhere,actWhatBefore,LSKA2,LSKA2,none
58,LSKA2-2191,"In the video, which was the person behind firs...",food,objFirstChoose,LSKA2,LSKA2,none
59,LSKA2-2192,"Of all the items they went on the side of, was...",food,objFirstChoose,LSKA2,LSKA2,none


In [83]:
total.to_csv('../../exports/%s/other_formats/Total-question.csv' % d_path, index = None)

In [66]:
len(train['key'].values), len(set(list(train['key'].values)))

(1718292, 1718292)

In [67]:
train.shape

(1718292, 7)

In [84]:
ba = train[(train['answer'] == 'before') | (train['answer'] == 'after')]
ba.shape

(54278, 7)

In [69]:
with open('../../exports/dataset/ids_to_del_obj.txt', 'rb') as f:
    obj = json.load(f)
with open('../../exports/dataset/ids_to_del_relact.txt', 'rb') as f:
     relact = json.load(f)

In [72]:
with open('../../exports/dataset/ids_to_del_everything.txt', 'rb') as f:
     problematic_ids = json.load(f)


In [73]:
ba_dropped = ba.drop(ba[ba.key.isin(problematic_ids)].index, inplace=True)
print(ba_dropped.shape[0])

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


AttributeError: 'NoneType' object has no attribute 'shape'

In [77]:
len(ba['question'].unique())


120410

In [85]:
len(ba['question'].unique())


27319

In [91]:
total.shape[0] - len(problematic_ids)

2270101

# novel compo

In [6]:
train_novel_comp = pickleLoad('../../exports/dataset/metrics/balanced/train/illegal_comp.pkl')
test_novel_comp = pickleLoad('../../exports/dataset/metrics/balanced/test/illegal_comp.pkl')

In [24]:
# Fixing and getting rid of deleted keys


YSKX3 <class 'list'> ['YSKX3-8', 'YSKX3-9', 'YSKX3-11', 'YSKX3-13', 'YSKX3-461', 'YSKX3-524', 'YSKX3-561', 'YSKX3-632', 'YSKX3-640', 'YSKX3-719', 'YSKX3-871', 'YSKX3-896', 'YSKX3-1119', 'YSKX3-1129', 'YSKX3-1155', 'YSKX3-1179', 'YSKX3-1180', 'YSKX3-1181', 'YSKX3-1182', 'YSKX3-1183', 'YSKX3-1184', 'YSKX3-1185', 'YSKX3-1186', 'YSKX3-1187', 'YSKX3-1188', 'YSKX3-1189', 'YSKX3-1190', 'YSKX3-1191', 'YSKX3-1192', 'YSKX3-1193', 'YSKX3-1194', 'YSKX3-1195', 'YSKX3-1196', 'YSKX3-1197', 'YSKX3-1198', 'YSKX3-1200', 'YSKX3-1201', 'YSKX3-1202', 'YSKX3-1203', 'YSKX3-1204', 'YSKX3-1205', 'YSKX3-1207', 'YSKX3-1208', 'YSKX3-1209', 'YSKX3-1210', 'YSKX3-1211', 'YSKX3-1212', 'YSKX3-1213', 'YSKX3-1214', 'YSKX3-1215', 'YSKX3-1216', 'YSKX3-1217', 'YSKX3-1218', 'YSKX3-1219', 'YSKX3-1220', 'YSKX3-1221', 'YSKX3-1222', 'YSKX3-1251', 'YSKX3-1255', 'YSKX3-1280', 'YSKX3-1294', 'YSKX3-1299', 'YSKX3-1300', 'YSKX3-1690', 'YSKX3-2547', 'YSKX3-2826', 'YSKX3-2849', 'YSKX3-2876', 'YSKX3-3128', 'YSKX3-3129', 'YSKX3-3130', 'Y

In [7]:
# Testing to CSV
#46GP8, N11GT, KRF68, MJO7C, S6MPZ

def toDataframeCompo(v_id, QA, compo):
    df = pd.DataFrame.from_dict(QA, orient="index")
    df['vid_id'] = v_id
    #df.index = v_id
    df['gif_name'] = v_id
    #df = df.drop(columns=['attributes', 'global', 'local', 'steps', 'metrics'])
    
    # NEW!
    #df = df.drop(columns=['attributes', 'global', 'id', 'local', 'steps', 'metrics', 'program', 'sg_grounding'])
    #df = df.reset_index()
    df['index'] = df.index
    df = df.rename(columns={'index': 'key'})
    #df = df.rename(columns={'id': 'key'})
    
    #df.set_index("key", inplace=True)
    # delete ones in compo
    if v_id in compo:
        to_del = list(set(compo[v_id]))
        
        
        
        df.drop(to_del, inplace=True, errors='ignore')
        #for i in to_del:
        #    print(i)
        #    print('YSKX3-11210' in df.key.values)
        #    df.drop(i, inplace=True)

    #print()
    #print(df.shape)
    #print(len(to_del))
    #print(df.shape)
    #if (df.shape[0] > 0):
    #    print(df.shape)
    return df


def toTGIFCompo(d_path, group, bal_txt):
    dataframes = []
    if group == 'train':
        our_data = train_stsgs
        compo = train_novel_comp
    if group == 'test':
        our_data = test_stsgs
        compo = test_novel_comp
        
    cnt = 0
    for v_id in our_data:
        if cnt % 100 == 0:
            print(cnt)
        cnt = cnt + 1
        QA = getQA(d_path, group, bal_txt, v_id)
        #print('len QA', len(QA))
        df = toDataframeCompo(v_id, QA, compo)
        #print(df.columns)
        # if it's empty continue
        if 'attributes' not in df.columns:
            continue
        df = df.drop(columns=['attributes', 'global', 'local', 'steps', 'metrics', 'program', 'sg_grounding'])
        df['description'] = "none"
        dataframes.append(df)
    total = pd.concat(dataframes)
    print('saving to ', '../../exports/%s/other_formats/%s-%s-tgif-compo.csv' % (d_path, group, bal_txt))
    total.to_csv('../../exports/%s/other_formats/%s-%s-tgif-compo.csv' % (d_path, group, bal_txt), index = None)
    return total


In [8]:
train_compo = toTGIFCompo('dataset', 'train', 'balanced')

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
saving to  ../../exports/dataset/other_formats/train-balanced-tgif-compo.csv


In [12]:
test_compo = toTGIFCompo('dataset', 'test', 'balanced')

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
saving to  ../../exports/dataset/other_formats/test-balanced-tgif-compo.csv


In [13]:
# load the pickles
# drop other columns
# attach together

d_path = 'dataset'
training = pd.read_csv('../../exports/%s/other_formats/train-balanced-tgif-compo.csv' % d_path)
testing = pd.read_csv('../../exports/%s/other_formats/test-balanced-tgif-compo.csv' % d_path)

In [14]:
total = pd.concat([training, testing])
total

,question,answer,id,vid_id,gif_name,key,description
0,"Of all the items they touched, was the first o...",food,46GP8-893,46GP8,46GP8,46GP8-893,none
1,"Which were they behind last, some food or a wi...",window,46GP8-898,46GP8,46GP8,46GP8-898,none
2,"In the video, of all the items they went behin...",window,46GP8-899,46GP8,46GP8,46GP8-899,none
3,"In the video, did they interact with a window?",Yes,46GP8-5,46GP8,46GP8,46GP8-5,none
4,Which object did the person interact with afte...,window,46GP8-388,46GP8,46GP8,46GP8-388,none
...,...,...,...,...,...,...,...
90936,Was the first thing they went on the side of t...,Yes,LSKA2-813,LSKA2,LSKA2,LSKA2-813,none
90937,"In the video, was the object they were behind ...",Yes,LSKA2-832,LSKA2,LSKA2,LSKA2-832,none
90938,"In the video, which was the person behind firs...",food,LSKA2-2191,LSKA2,LSKA2,LSKA2-2191,none
90939,"Of all the items they went on the side of, was...",food,LSKA2-2192,LSKA2,LSKA2,LSKA2-2192,none


In [15]:
total.to_csv('../../exports/%s/other_formats/Total-question-compo.csv' % d_path, index = None)

# By Compositional Steps

In [16]:
# Testing to CSV
#46GP8, N11GT, KRF68, MJO7C, S6MPZ

def toDataframeSteps(v_id, QA, steps):
    df = pd.DataFrame.from_dict(QA, orient="index")
    df['vid_id'] = v_id
    #df.index = v_id
    df['gif_name'] = v_id
    #df = df.drop(columns=['attributes', 'global', 'local', 'steps', 'metrics'])
    
    # new!
   # df = df.drop(columns=['attributes', 'global', 'id', 'local', 'steps', 'metrics', 'program', 'sg_grounding'])
    df['index'] = df.index
    df = df.rename(columns={'index': 'key'})
    
    #df = df.rename(columns={'id': 'key'})
    df['description'] = "none"
    
    if v_id in steps:
        to_del = list(set(steps[v_id]))
        df.drop(to_del, inplace=True, errors='ignore')
    
    return df


def toTGIFSteps(d_path, group, bal_txt, title, steps):
    dataframes = []
    if group == 'train':
        our_data = train_stsgs
    if group == 'test':
        our_data = test_stsgs
        
    cnt = 0
    for v_id in our_data:
        if cnt % 1000 == 0:
            print(cnt)
        cnt = cnt + 1
        QA = getQA(d_path, group, bal_txt, v_id)
        #print('len QA', len(QA))
        df = toDataframeSteps(v_id, QA, steps)
        if 'attributes' not in df.columns:
            continue
        df = df.drop(columns=['attributes', 'global', 'local', 'steps', 'metrics', 'program', 'sg_grounding'])
        dataframes.append(df)
    total = pd.concat(dataframes)
    total.to_csv('../../exports/%s/other_formats/%s-%s-tgif-%s.csv' % (d_path, group, bal_txt, title), index = None)
    return total


In [17]:
#train_steps_glob = pickleLoad('../exports/dataset/metrics/templ_balanced/train/steps_by_glob.pkl')
#test_steps_glob = pickleLoad('../exports/dataset/metrics/balanced/test/steps_by_glob.pkl')
train_steps_templ = pickleLoad('../../exports/dataset/metrics/balanced/train/steps_by_templ.pkl')
test_steps_templ = pickleLoad('../../exports/dataset/metrics/balanced/test/steps_by_templ.pkl')

In [18]:
for i in train_steps_templ:
    print(i, len(train_steps_templ[i]))
    j = train_steps_templ[i]
    for k in j:
        print(k)
        for l in j[k]:
            print(l)
            break
        break

less 7786
46GP8
46GP8-5
more 6519
46GP8
46GP8-495


In [19]:
# so note to self - want to pass in the part of global to delete
d_path = 'dataset'
#glob_split_tr = toTGIFSteps(d_path, 'train', 'balanced', 'steps_glob', train_steps_glob['more'])


In [20]:
#glob_split_te = toTGIFSteps(d_path, 'test', 'balanced', 'steps_glob', test_steps_glob['less'])

In [21]:
for i in train_steps_templ:
    print(i)

less
more


In [22]:
templ_split_tr = 0
toTGIFSteps(d_path, 'train', 'balanced', 'steps_templ', train_steps_templ['more'])

0
1000
2000
3000
4000
5000
6000
7000


,question,answer,id,vid_id,gif_name,key,description
46GP8-893,"Of all the items they touched, was the first o...",food,46GP8-893,46GP8,46GP8,46GP8-893,none
46GP8-898,"Which were they behind last, some food or a wi...",window,46GP8-898,46GP8,46GP8,46GP8-898,none
46GP8-899,"In the video, of all the items they went behin...",window,46GP8-899,46GP8,46GP8,46GP8-899,none
46GP8-5,"In the video, did they interact with a window?",Yes,46GP8-5,46GP8,46GP8,46GP8-5,none
46GP8-388,Which object did the person interact with afte...,window,46GP8-388,46GP8,46GP8,46GP8-388,none
...,...,...,...,...,...,...,...
PG1ID-1791,What did they watch while holding the object t...,laptop,PG1ID-1791,PG1ID,PG1ID,PG1ID-1791,none
PG1ID-1790,What did they watch while holding the thing th...,laptop,PG1ID-1790,PG1ID,PG1ID,PG1ID-1790,none
PG1ID-972,While watching a laptop or something on a lapt...,Yes,PG1ID-972,PG1ID,PG1ID,PG1ID-972,none
PG1ID-1789,Which object did the person watch while taking...,laptop,PG1ID-1789,PG1ID,PG1ID,PG1ID-1789,none


In [23]:
templ_split_te = 0
toTGIFSteps(d_path, 'test', 'balanced', 'steps_templ', test_steps_templ['less'])

0
1000


,question,answer,id,vid_id,gif_name,key,description
YSKX3-1690,After tidying up the object they were in front...,No,YSKX3-1690,YSKX3,YSKX3,YSKX3-1690,none
YSKX3-2547,Was the person touching something while throwi...,Yes,YSKX3-2547,YSKX3,YSKX3,YSKX3-2547,none
YSKX3-2826,Did they tidy anything before taking the thing...,Yes,YSKX3-2826,YSKX3,YSKX3,YSKX3-2826,none
YSKX3-2849,"Before holding the thing they put down, was th...",Yes,YSKX3-2849,YSKX3,YSKX3,YSKX3-2849,none
YSKX3-2876,While taking the thing they put down from some...,Yes,YSKX3-2876,YSKX3,YSKX3,YSKX3-2876,none
...,...,...,...,...,...,...,...
F1LTO-3352,Was the person closing something before or aft...,after,F1LTO-3352,F1LTO,F1LTO,F1LTO-3352,none
F1LTO-3379,Was the person closing the last thing they tou...,after,F1LTO-3379,F1LTO,F1LTO,F1LTO-3379,none
D8K7T-1383,Were they interacting with a pillow while thro...,No,D8K7T-1383,D8K7T,D8K7T,D8K7T-1383,none
D8K7T-1702,While throwing the object they were carrying s...,No,D8K7T-1702,D8K7T,D8K7T,D8K7T-1702,none


In [24]:

steps_tr = pd.read_csv('../../exports/%s/other_formats/train-balanced-tgif-steps_templ.csv' % d_path)

In [25]:
steps_tr.columns

Index(['question', 'answer', 'id', 'vid_id', 'gif_name', 'key', 'description'], dtype='object')

In [26]:
x = steps_tr['key'].values

In [27]:

training = pd.read_csv('../../exports/%s/other_formats/train-balanced-tgif-steps_templ.csv' % d_path)
testing = pd.read_csv('../../exports/%s/other_formats/test-balanced-tgif-steps_templ.csv' % d_path)

In [28]:
len(training['vid_id'].unique())

7786

In [29]:
total = pd.concat([training, testing])
total

,question,answer,id,vid_id,gif_name,key,description
0,"Of all the items they touched, was the first o...",food,46GP8-893,46GP8,46GP8,46GP8-893,none
1,"Which were they behind last, some food or a wi...",window,46GP8-898,46GP8,46GP8,46GP8-898,none
2,"In the video, of all the items they went behin...",window,46GP8-899,46GP8,46GP8,46GP8-899,none
3,"In the video, did they interact with a window?",Yes,46GP8-5,46GP8,46GP8,46GP8-5,none
4,Which object did the person interact with afte...,window,46GP8-388,46GP8,46GP8,46GP8-388,none
...,...,...,...,...,...,...,...
89641,Was the person closing something before or aft...,after,F1LTO-3352,F1LTO,F1LTO,F1LTO-3352,none
89642,Was the person closing the last thing they tou...,after,F1LTO-3379,F1LTO,F1LTO,F1LTO-3379,none
89643,Were they interacting with a pillow while thro...,No,D8K7T-1383,D8K7T,D8K7T,D8K7T-1383,none
89644,While throwing the object they were carrying s...,No,D8K7T-1702,D8K7T,D8K7T,D8K7T-1702,none


In [30]:
total.to_csv('../../exports/%s/other_formats/Total-question-steps_templ.csv' % d_path, index = None)

In [11]:
# Testing to CSV
#46GP8, N11GT, KRF68, MJO7C, S6MPZ

def toDataframeLength(v_id, QA):
    df = pd.DataFrame.from_dict(QA, orient="index")
    df['vid_id'] = v_id
    #df.index = v_id
    df['gif_name'] = v_id
    df = df.drop(columns=['attributes', 'global', 'local', 'steps', 'metrics'])
    df = df.rename(columns={'id': 'key'})
    
    return df


def toTGIFLength(d_path, group, bal_txt, title, length):
    dataframes = []
    if group == 'train':
        our_data = train_stsgs
    if group == 'test':
        our_data = test_stsgs
        
    cnt = 0
    for v_id in our_data:
        if cnt % 1000 == 0:
            print(cnt)
        cnt = cnt + 1
        if v_id in length:
            continue
        QA = getQA(d_path, group, bal_txt, v_id)
        #print('len QA', len(QA))
        df = toDataframeLength(v_id, QA)
        dataframes.append(df)
    total = pd.concat(dataframes)
    total.to_csv('../exports/%s/other_formats/%s-%s-tgif-%s.csv' % (d_path, group, bal_txt, title), index = None)
    return total


In [ ]:
train_length = pickleLoad('../exports/dataset/metrics/balanced/train/lengths.pkl')
test_length = pickleLoad('../exports/dataset/metrics/balanced/test/lengths.pkl')

In [ ]:
lengths_tr = toTGIFLength(d_path, 'train', 'balanced', 'lengths', train_length['more'])
lengths_te = toTGIFLength(d_path, 'test', 'balanced', 'lengths', test_length['less'])

# Reduce test set b/c hme is a baby

In [6]:
d_path = 'dataset'


# attach together
training = pd.read_csv('../exports/%s/other_formats/train-balanced-tgif.csv' % d_path)
testing = pd.read_csv('../exports/%s/other_formats/test-balanced-tgif.csv' % d_path)

In [7]:
compo_test = pd.read_csv('../exports/%s/other_formats/test-balanced-tgif-compo.csv' % d_path)
#lengths_test = pd.read_csv('../exports/%s/other_formats/test-balanced-tgif-lengths.csv' % d_path)
steps_test = pd.read_csv('../exports/%s/other_formats/test-balanced-tgif-steps_templ.csv' % d_path)

In [ ]:
steps_sample = steps_test.sample(n=200000).sort_index()

In [ ]:
len(steps_sample)

In [ ]:
len(steps_sample)
steps_sample.to_csv('../exports/%s/other_formats/%s-%s-tgif-steps_sample.csv' % ('dataset', 'test', 'balanced'), index = None)

In [ ]:
to_keep

In [ ]:
with open('../data/relactindir.txt', 'rb') as f:
    to_keep = json.load(f)

In [ ]:
to_keep_df = testing.loc[testing['key'].isin(to_keep)]

In [ ]:
len(to_keep_df)

In [ ]:

to_keep_df.to_csv('../exports/%s/other_formats/%s-%s-tgif-rel_act.csv' % ('dataset', 'test', 'balanced'), index = None)




In [ ]:
lengths_sample = lengths_test.sample(n=100000).sort_index()

In [ ]:
len(lengths_sample)
lengths_sample.to_csv('../exports/%s/other_formats/%s-%s-tgif-lengths_sample.csv' % ('dataset', 'test', 'balanced'), index = None)

In [9]:
#testing_no_relact = testing[testing['']]

test_sample = testing.sample(n=100000).sort_index()

In [15]:
def getAllCat():
    # so here, initalize all the categories that matter
    1+1

def splitByVideo(df):
    by_v = {}
    for q_id in df['key']:
        v = q_id[:5]
        if v not in by_v:
            by_v[v] = []
        by_v[v].append(q_id)
        
    return by_v
    
def containsAllCat(by_v):
    globs = getGlobDict()
    sems = getSemDict()
    structs = getStructDict()
    
    steps = {
        1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 
    }
    
    for v_id in by_v:
        QA = getQA('dataset', 'test', 'balanced', v_id)
        
        for q_id in QA:
            q = QA[q_id]
            glob = getGlob(q)
            sem = getSemantic(q)
            struct = getStructural(q)
            step = q['steps']
            tp = getType(q)
            
            for g in glob:
                globs[g][tp] += 1
            sems[sem][tp] += 1
            structs[struct][tp] += 1
            
            steps[step] += 1
            
    
    return globs, sems, structs, steps

by_v = splitByVideo(test_sample)


glob, sem, struct, steps = containsAllCat(by_v)



In [16]:
glob

{'obj-rel': {'binary': 174462, 'open': 292951},
 'superlative': {'binary': 93081, 'open': 63530},
 'exists': {'binary': 89139, 'open': 0},
 'count': {'binary': 0, 'open': 0},
 'duration-comparison': {'binary': 13007, 'open': 358},
 'obj-act': {'binary': 63, 'open': 0},
 'rel-act': {'binary': 38053, 'open': 0},
 'sequencing': {'binary': 39039, 'open': 416},
 'action-recognition': {'binary': 0, 'open': 1106}}

In [17]:
sem

{'object': {'binary': 144809, 'open': 295525},
 'relation': {'binary': 97481, 'open': 0},
 'action': {'binary': 66903, 'open': 1464}}

In [18]:
struct

{'query': {'binary': 0, 'open': 296989},
 'compare': {'binary': 90162, 'open': 0},
 'choose': {'binary': 94435, 'open': 0},
 'verify': {'binary': 94883, 'open': 0},
 'logic': {'binary': 29713, 'open': 0}}

In [19]:
steps

{1: 249,
 2: 15460,
 3: 14686,
 4: 149959,
 5: 269348,
 6: 95124,
 7: 51320,
 8: 10026,
 9: 10,
 10: 0}

In [10]:
test_sample.to_csv('../exports/%s/other_formats/%s-%s-tgif-all_sample.csv' % ('dataset', 'test', 'balanced'), index = None)

# stuff to initalize for splitting test set

In [12]:
templ2global = {
    'objExists': ['exists'],
    'objRelExists': ['exists', 'obj-rel'],
    'relExists': ['exists'],
    'actExists': ['exists'],
    'andObjRelExists': ['exists', 'obj-rel'],
    'xorObjRelExists': ['exists', 'obj-rel'],
    'objWhatGeneral': [],
    'objWhat': ['obj-rel'],
    'objWhatChoose': ['obj-rel'],
    'actWhatAfterAll': ['sequencing', 'action-recognition'],
    'actWhatBefore': ['sequencing', 'action-recognition'],
    'objFirst': ['superlative', 'obj-rel'],
    'objFirstChoose': ['superlative', 'obj-rel'],
    'objFirstVerify': ['superlative', 'obj-rel'],
    'relFirst': ['superlative', 'obj-rel'],
    'actFirst': ['superlative', 'action-recognition'],
    'objLast': ['superlative', 'obj-rel'],
    'objLastChoose': ['superlative', 'obj-rel'],
    'objLastVerify': ['superlative', 'obj-rel'],
    'relLast': ['superlative', 'obj-rel'],
    'actLast': ['superlative', 'action-recognition'],
    'actCount': ['count'],
    'actCountChooseMore': ['count'],
    'actCountChooseFewer': ['count'],
    'actLengthLongerChoose': ['duration-comparison'],
    'actLengthShorterChoose': ['duration-comparison'],
    'actLengthLongerVerify': ['duration-comparison'],
    'actLengthShorterVerify': ['duration-comparison'],
    'actLongest': ['superlative', 'duration-comparison'],
    'actShortest': ['superlative', 'duration-comparison'],
    'actTime': ['sequencing'],
    'relTime': ['rel-act'],
    'objTime': ['obj-act'],
}

In [13]:
def getType(q):
    tp = q['attributes']['structural']
    if tp != 'query':
        return 'binary'
    else:
        return 'open'
    
def getStructural(q):
    tp = q['attributes']['structural']
    
    return tp
    
def getSemantic(q):
    tp = q['attributes']['semantic']
    if tp == 'objrel':
        return 'relation'
    elif tp in ['object', 'relation', 'action']:
        return tp
    else:
        return 'none'
    
def getGlob(q):
    tp = q['attributes']['type']
    return templ2global[tp]

In [14]:
def getSemDict():
    sem = {'object': {'binary': 0, 'open': 0},
           'relation': {'binary': 0, 'open': 0},
           'action': {'binary': 0, 'open': 0},}
    return sem
    
def getGlobDict():
    #glob = {'obj-rel': 209,
        #     'superlative': 51,
       #      'exists': 206,
      #       'count': 20,
     #        'duration-comparison': 7,
     #        'obj-act': 10,
     #        'rel-act': 11,
     #        'sequencing': 3,
    #         'action-recognition': 1}
    glob = {'obj-rel': {'binary': 0, 'open': 0},
             'superlative': {'binary': 0, 'open': 0},
             'exists': {'binary': 0, 'open': 0},
             'count':{'binary': 0, 'open': 0},
             'duration-comparison': {'binary': 0, 'open': 0},
             'obj-act': {'binary': 0, 'open': 0},
             'rel-act': {'binary': 0, 'open': 0},
             'sequencing': {'binary': 0, 'open': 0},
             'action-recognition': {'binary': 0, 'open': 0},}
    return glob
    
def getStructDict():
    struct = {'query': {'binary': 0, 'open': 0},
              'compare': {'binary': 0, 'open': 0},
              'choose': {'binary': 0, 'open': 0},
              'verify': {'binary': 0, 'open': 0},
              'logic': {'binary': 0, 'open': 0},}
    #struct = {'query': 316, 'compare': 266, 'choose': 55, 'verify': 107, 'logic': 53}
    return struct


#  Old stuff

In [ ]:
# Note, putting all in the same category
# [{"answer":"man","category_id":9,"id":0,"question":"who drives down the road in an audi?","video_id":0}, {...}, ...]
def toMSVDQA(our_data):
    questions = []
    
    for v_id in our_data:
        for q_id in QA[v_id]:
            q = our_data[v_id][q_id]
            
            new_q = {
                'answer': q['answer'],
                'category_id':9, # TODO: not sure what this is
                'id': q_id,
                'question': q['question'],
                'video_id': v_id
            }
            
            questions.append(new_q)
    return questions
    

In [ ]:
# [{"answer":"animal","id":0,"question":"what is chewing on a nut?","video_id":1},
def toMSRVTTQA(our_data):
    questions = []
    
    for v_id in our_data:
        for q_id in QA[v_id]:
            q = our_data[v_id][q_id]
            
            new_q = {
                'answer': q['answer'],
                'id': q_id,
                'question': q['question'],
                'video_id': v_id
            }
            
            questions.append(new_q)
    return questions
    

In [ ]:
def toDataframe(our_data):
    dataframes = []
    count = 0
    for v_id in our_data:
        count = count + 1
        if count > 5: 
            break
        print(v_id)
        print("making a dataframe")
        df = pd.DataFrame.from_dict(our_data, orient="index")
        #df.to_csv(r'../exports/all_videos_200-300_tgif_format.csv', index = None)
        print("adding vid id")
        df['vid_id'] = v_id
        print("appending")
        dataframes.append(df),
    total = pd.concat(dataframes)
    return total
    

def toTGIFQA(df):
    #df = toDataframe(our_data)
    # drop everything not important
    df['gif_name'] = df['vid_id']
    df.rename(columns={'id': 'key'})
    df.drop(columns=['attributes', 'global', 'local', 'steps', 'metrics'])
    print("AFTER DROP")
    print(df)
    return df
    

# Get templ balancd

In [ ]:

def toTGIFTemplBalanced(d_path, group, bal_txt):
    dataframes = []
    if group == 'train':
        our_data = train_stsgs
    if group == 'test':
        our_data = test_stsgs
        
    #if balanced: 
    #    bal_txt = 'balanced'
    #else:
    #    bal_txt = 'all'
        
    # for templ balanced
    #bal_txt = "templ_balanced"
        
    cnt = 0
    for v_id in our_data:
        if cnt % 100 == 0:
            print(cnt)
        cnt = cnt + 1
        QA = getQA(d_path, group, bal_txt, v_id)
        df = toDataframe(v_id, QA)
        dataframes.append(df)
    total = pd.concat(dataframes)
    total.to_csv('../exports/%s/other_formats/%s-%s-tgif.csv' % (d_path, group, bal_txt), index = None)
    return total


In [ ]:
toTGIFTemplBalanced('dataset-before-11-4', 'test', 'templ_balanced')

In [ ]:
toTGIFTemplBalanced('dataset-before-11-4', 'train', 'templ_balanced')

# Length

# Start Query Questions

In [ ]:
# Testing to CSV
#46GP8, N11GT, KRF68, MJO7C, S6MPZ

def toDataframeQuery(v_id, QA):
    df = pd.DataFrame.from_dict(QA, orient="index")
    df['vid_id'] = v_id
    #df.index = v_id
    df['gif_name'] = v_id
    df = df.drop(columns=['attributes', 'global', 'local', 'steps', 'metrics'])
    df = df.rename(columns={'id': 'key'})
    
    #df.set_index("key", inplace=True)
    # delete ones in compo
    #if v_id not in compo:
   #     return df
   # to_del = list(set(compo[v_id]))
    
    to_del = []
    query = ['objWhat', 'objFirst', 'objLast', 'relLast']
    for q_id in QA:
        t = QA[q_id]['attributes']['type']
        #print(t)
        if t in query:
            continue
            
        to_del.append(q_id)
    
    #print()
    #print(df.shape)
    #print(len(to_del))
    df.drop(to_del, inplace=True)
    #print(df.shape)
    #if (df.shape[0] > 0):
    #    print(df.shape)
    return df


def toTGIFQuery(d_path, group, balanced):
    dataframes = []
    if group == 'train':
        our_data = train_stsgs
        compo = train_novel_comp
    if group == 'test':
        our_data = test_stsgs
        compo = test_novel_comp
        
    if balanced: 
        bal_txt = 'balanced'
    else:
        bal_txt = 'all'
        
    cnt = 0
    for v_id in our_data:
        if cnt % 100 == 0:
            print(cnt)
        cnt = cnt + 1
        QA = getQA(d_path, group, bal_txt, v_id)
        #print('len QA', len(QA))
        df = toDataframeQuery(v_id, QA)
        dataframes.append(df)
    total = pd.concat(dataframes)
    total.to_csv('../exports/%s/other_formats/%s-%s-tgif-query.csv' % (d_path, group, bal_txt), index = None)
    return total


In [ ]:
test_query = toTGIFQuery('dataset', 'test', True)

In [ ]:
test_query = toTGIFQuery('dataset', 'train', True)

607120 +  1647710

# Get TOTAL

In [ ]:
#training_edited = training.drop(columns=['question', 'answer'])
training_edited = training

In [ ]:
#testing_edited = testing.drop(columns=['question', 'answer'])
testing_edited = testing

In [ ]:
total = pd.read_csv('../exports/dataset/other_formats/Total-question.csv')

In [ ]:
total.columns

In [ ]:
total = pd.read_csv('../exports/dataset/other_formats/Total-question.csv')

In [ ]:
def getVIDMapping(videos):
    str2num = {}
    num2str = {}
    
    str_ids = set(videos['gif_name'])
    
    count = 0
    for str_id in str_ids:
        str2num[str_id] = count
        num2str[count] = str_id
        
        count = count + 1
        
    return str2num, num2str
    
str2num, num2str = getVIDMapping(total)

In [ ]:
# do json not pickle

with open('../data/strID2numID.json', 'w+') as json_file: #CHANGED
    json.dump(str2num, json_file)
    
with open('../data/numID2strID.json', 'w+') as json_file: #CHANGED
    json.dump(num2str, json_file)
#pickleDump(str2num, '../data/strID2numID')
#pickleDum(num2str, '../data/strID2numID')

In [ ]:
toTGIF('dataset', 'test', False)

In [ ]:
b_test = toTGIF('dataset', 'test', True)

In [ ]:
QA = getQA('dataset', 'test', 'balanced', 'YSKX3')
print(len(QA))

inQA = 0
notinQA = 0
for i in test_novel_comp['YSKX3']:
    if i in QA:
        inQA = inQA + 1
    else:
        notinQA = notinQA + 1

inQA, notinQA, len(set(test_novel_comp['YSKX3']))

In [ ]:
train_novel_comp['46GP8']

In [ ]:
for i in test_compo['question']:
    print(i)

In [ ]:
print(len(test_steps_templ['less']))

In [ ]:
for question in templ_split_te['question']:
    print(question)

# end query questions

In [ ]:
import numpy as np
b_test_perm = b_test.iloc[np.random.permutation(len(b_test))]

In [ ]:
questions = list(b_test_perm['question'])

In [ ]:
questions[0]

In [ ]:
b_test2 = pd.read_csv('../exports/dataset/other_formats/train-balanced-tgif.csv')

In [ ]:
b_test2_perm = b_test2.iloc[np.random.permutation(len(b_test2))]

In [ ]:
questions2 = list(b_test2_perm['question'])

In [ ]:
toTGIF('dataset', 'test', True)

In [ ]:
toTGIF('dataset', 'train', True)

In [ ]:
msvd_translation = toMSVDQA(QA)

In [ ]:
len(msvd_translation) * 100

In [ ]:
msrvtt_translation = toMSRVTTQA(QA)

In [ ]:
len(msvd_translation) * 100

In [ ]:
dataframe = toDataframe(QA['46GP8'])
dataframe = toTGIFQA(dataframe)
dataframe

In [ ]:
for i in QA:
    print(i)
    break

In [ ]:
# Need to add in v_id column with the v_id for all of them
# gif_name, = v_id (idk what the difference is with this and vid_id, so im going to do both the same)
# question, answer, vid_id, 
# key = ?

In [ ]:
len(QA['3CPVF']), len(QA['QIQXR'])

In [ ]:
#'3CPVF': 23415, 'QIQXR': 7186, 'SXYLN': 30109, 'I6B5D': 874, 'MJX48': 15637

In [ ]:
false = False
true = True

In [ ]:
a = {"46GP8-321": {"question": "Was the person closing anything while cooking something?", "answer": "No", "attributes": {"type": "relExists", "structural": "verify", "semantic": "relation", "ans_type": "binary", "video_id": "46GP8"}, "global": "exists", "local": "v001", "steps": 3, "id": "46GP8-321", "metrics": {"novel_comp": [false, false, false, false, false], "indirects": [false, false, false, true], "direct_args": "v001", "direct_time": ["while", "c147"]}}, "46GP8-564": {"question": "Which object did the person touch after cooking something?", "answer": "window", "attributes": {"type": "objWhat", "structural": "query", "semantic": "object", "ans_type": "open", "video_id": "46GP8"}, "global": "what", "local": "r22", "steps": 4, "id": "46GP8-564", "metrics": {"novel_comp": [false, false, false, false, false], "indirects": [false, false, false, true], "direct_args": "r22", "direct_time": ["after", "c147"]}}, "46GP8-565": {"question": "In the video, what was the person touching?", "answer": "window", "attributes": {"type": "objWhat", "structural": "query", "semantic": "object", "ans_type": "open", "video_id": "46GP8"}, "global": "what", "local": "r22", "steps": 2, "id": "46GP8-565", "metrics": {"novel_comp": [false, false, false, false, false], "indirects": [false, false, false, false], "direct_args": "r22", "direct_time": ["all", ""]}}, "46GP8-566": {"question": "What was the person behind after cooking something?", "answer": "window", "attributes": {"type": "objWhat", "structural": "query", "semantic": "object", "ans_type": "open", "video_id": "46GP8"}, "global": "what", "local": "r7", "steps": 4, "id": "46GP8-566", "metrics": {"novel_comp": [false, false, false, false, false], "indirects": [false, false, false, true], "direct_args": "r7", "direct_time": ["after", "c147"]}}, "46GP8-567": {"question": "Which object did the person go above?", "answer": "food", "attributes": {"type": "objWhat", "structural": "query", "semantic": "object", "ans_type": "open", "video_id": "46GP8"}, "global": "what", "local": "r4", "steps": 2, "id": "46GP8-567", "metrics": {"novel_comp": [false, false, false, false, false], "indirects": [false, false, false, false], "direct_args": "r4", "direct_time": ["all", ""]}}, "46GP8-568": {"question": "Which object were they on the side of after cooking something?", "answer": "window", "attributes": {"type": "objWhat", "structural": "query", "semantic": "object", "ans_type": "open", "video_id": "46GP8"}, "global": "what", "local": "r8", "steps": 4, "id": "46GP8-568", "metrics": {"novel_comp": [false, false, false, false, false], "indirects": [false, false, false, true], "direct_args": "r8", "direct_time": ["after", "c147"]}}, "46GP8-569": {"question": "In the video, what was the person on the side of?", "answer": "window", "attributes": {"type": "objWhat", "structural": "query", "semantic": "object", "ans_type": "open", "video_id": "46GP8"}, "global": "what", "local": "r8", "steps": 2, "id": "46GP8-569", "metrics": {"novel_comp": [false, false, false, false, false], "indirects": [false, false, false, false], "direct_args": "r8", "direct_time": ["all", ""]}}, "46GP8-571": {"question": "After cooking something, which object were they watching?", "answer": "window", "attributes": {"type": "objWhat", "structural": "query", "semantic": "object", "ans_type": "open", "video_id": "46GP8"}, "global": "what", "local": "v031", "steps": 4, "id": "46GP8-571", "metrics": {"novel_comp": [false, false, false, false, false], "indirects": [false, false, false, true], "direct_args": "v031", "direct_time": ["after", "c147"]}}, "46GP8-572": {"question": "In the video, which object did the person watch?", "answer": "window", "attributes": {"type": "objWhat", "structural": "query", "semantic": "object", "ans_type": "open", "video_id": "46GP8"}, "global": "what", "local": "v031", "steps": 2, "id": "46GP8-572", "metrics": {"novel_comp": [false, false, false, false, false], "indirects": [false, false, false, false], "direct_args": "v031", "direct_time": ["all", ""]}}, "46GP8-577": {"question": "Which did they go on the side of, some food or a window?", "answer": "window", "attributes": {"type": "objWhatChoose", "structural": "choose", "semantic": "object", "ans_type": "open", "video_id": "46GP8"}, "global": "what", "local": "r8", "steps": 3, "id": "46GP8-577", "metrics": {"novel_comp": [false, false, false, false, false], "indirects": [false, false, false, false], "direct_args": "r8-o17-o36", "direct_time": ["all", ""]}}, "46GP8-579": {"question": "In the video, of everything they went behind, what was the person behind first?", "answer": "food", "attributes": {"type": "objFirst", "structural": "query", "semantic": "object", "ans_type": "open", "video_id": "46GP8"}, "global": "first", "local": "r7", "steps": 2, "id": "46GP8-579", "metrics": {"novel_comp": [false, true, false, false, false], "indirects": [false, false, false, false], "direct_args": "r7", "direct_time": ["all", ""]}}, "46GP8-583": {"question": "Which object were they behind last?", "answer": "window", "attributes": {"type": "objLast", "structural": "query", "semantic": "object", "ans_type": "open", "video_id": "46GP8"}, "global": "last", "local": "r7", "steps": 2, "id": "46GP8-583", "metrics": {"novel_comp": [false, false, false, false, false], "indirects": [false, false, false, false], "direct_args": "r7", "direct_time": ["all", ""]}}, "46GP8-588": {"question": "What was the last thing the person was doing to some food?", "answer": "cooking", "attributes": {"type": "relLast", "structural": "query", "semantic": "relation", "ans_type": "open", "video_id": "46GP8"}, "global": "last", "local": "o17", "steps": 2, "id": "46GP8-588", "metrics": {"novel_comp": [false, false, false, false, false], "indirects": [false, false, false, false], "direct_args": "o17", "direct_time": ["all", ""]}}, "46GP8-595": {"question": "Did they cook anything before they watched outside of a window, or after they watched outside of a window?", "answer": "before", "attributes": {"type": "relTime", "structural": "compare", "semantic": "relation", "ans_type": "binary", "video_id": "46GP8"}, "global": "time", "local": "v002-c092", "steps": 5, "id": "46GP8-595", "metrics": {"novel_comp": [false, false, false, false, false], "indirects": [false, false, false, false], "direct_args": "v002-c092", "direct_time": ["all", ""]}}, "46GP8-599": {"question": "Were they touching the first thing they went behind before or after they watched outside of a window?", "answer": "before", "attributes": {"type": "objTime", "structural": "compare", "semantic": "object", "ans_type": "binary", "video_id": "46GP8"}, "global": "time", "local": "o17-c092", "steps": 6, "id": "46GP8-599", "metrics": {"novel_comp": [false, true, false, false, false], "indirects": [true, false, false, false], "direct_args": "o17-c092", "direct_time": ["all", ""]}}, "46GP8-602": {"question": "Were they touching the object they were touching before or after they cooked something?", "answer": "after", "attributes": {"type": "objTime", "structural": "compare", "semantic": "object", "ans_type": "binary", "video_id": "46GP8"}, "global": "time", "local": "o36-c147", "steps": 6, "id": "46GP8-602", "metrics": {"novel_comp": [false, false, false, false, false], "indirects": [true, false, false, false], "direct_args": "o36-c147", "direct_time": ["all", ""]}}, "46GP8-605": {"question": "What was the person interacting with after cooking something?", "answer": "window", "attributes": {"type": "objWhatGeneral", "structural": "query", "semantic": "object", "ans_type": "open", "video_id": "46GP8"}, "global": "what", "local": "object", "steps": 3, "id": "46GP8-605", "metrics": {"novel_comp": [false, false, false, false, false], "indirects": [false, false, false, true], "direct_args": "", "direct_time": ["after", "c147"]}}, "46GP8-607": {"question": "How many times did they watch outside of a window while cooking something?", "answer": 0, "attributes": {"type": "actCount", "structural": "query", "semantic": "action", "ans_type": "open", "video_id": "46GP8"}, "global": "count", "local": "c092", "steps": 3, "id": "46GP8-607", "metrics": {"novel_comp": [false, false, false, false, false], "indirects": [false, false, false, true], "direct_args": "c092", "direct_time": ["while", "c147"]}}, "46GP8-608": {"question": "While watching outside of a window, how many times were they cooking something?", "answer": 0, "attributes": {"type": "actCount", "structural": "query", "semantic": "action", "ans_type": "open", "video_id": "46GP8"}, "global": "count", "local": "c147", "steps": 3, "id": "46GP8-608", "metrics": {"novel_comp": [false, false, false, false, false], "indirects": [false, false, false, true], "direct_args": "c147", "direct_time": ["while", "c092"]}}}

In [ ]:
len(a)

# Get the sizes of each

In [ ]:
d_path = 'dataset'
training_b = pd.read_csv('../exports/%s/other_formats/train-balanced-tgif.csv' % d_path)
testing_b = pd.read_csv('../exports/%s/other_formats/test-balanced-tgif.csv' % d_path)

In [ ]:

training_c = pd.read_csv('../exports/%s/other_formats/train-balanced-tgif-compo.csv' % d_path)
testing_c = pd.read_csv('../exports/%s/other_formats/test-balanced-tgif-compo.csv' % d_path)

In [ ]:

training_s = pd.read_csv('../exports/%s/other_formats/train-balanced-tgif-steps_templ.csv' % d_path)
testing_s = pd.read_csv('../exports/%s/other_formats/test-balanced-tgif-steps_templ.csv' % d_path)

In [ ]:
print('Training set: ', training_b.shape[0])
print('Testing set: ', testing_b.shape[0])

print('Training set compo: ', training_c.shape[0])
print('Testing set compo: ', testing_c.shape[0])

print('Training set steps: ', training_s.shape[0])
print('Testing set steps: ', testing_s.shape[0])

# Why did test randomly generate wrogn keys

In [37]:
d_path = 'dataset_fixed'

In [38]:
# Testing to CSV
#46GP8, N11GT, KRF68, MJO7C, S6MPZ

def toDataframe(v_id, QA):
    df = pd.DataFrame.from_dict(QA, orient="index")
    df['vid_id'] = v_id
    #df.index = v_id
    df['gif_name'] = v_id
    df = df.drop(columns=['attributes', 'global', 'id', 'local', 'steps', 'metrics', 'program', 'sg_grounding'])
    df = df.reset_index()
    df = df.rename(columns={'index': 'key'})
    df['description'] = "none"
    return df


def toTGIF(d_path, group, bal_txt):
    dataframes = []
    if group == 'train':
        our_data = train_stsgs
    if group == 'test':
        our_data = test_stsgs
        
    cnt = 0
    for v_id in our_data:
        if cnt % 1000 == 0:
            print(cnt)
        cnt = cnt + 1
        QA = getQA(d_path, group, bal_txt, v_id)
        if len(QA) == 0: 
            continue
        df = toDataframe(v_id, QA)
        dataframes.append(df)
    total = pd.concat(dataframes)
    total.to_csv('../exports/%s/other_formats/%s-%s-tgif-updated.csv' % (d_path, group, bal_txt), index = None)
    return total


In [39]:

train = toTGIF(d_path, 'train', bal_txt)

0
1000
2000
3000
4000
5000
6000
7000


In [41]:
test = toTGIF(d_path, 'test', bal_txt)

0
1000


In [42]:
# load the pickles
# drop other columns
# attach together
training = pd.read_csv('../exports/%s/other_formats/train-balanced-tgif-updated.csv' % d_path)
testing = pd.read_csv('../exports/%s/other_formats/test-balanced-tgif-updated.csv' % d_path)
total = pd.concat([training, testing])
total

,key,question,answer,vid_id,gif_name,description
0,46GP8-2,Is there some food the person contacts?,Yes,46GP8,46GP8,none
1,46GP8-6,Were they touching a window?,Yes,46GP8,46GP8,none
2,46GP8-36,Is there a table the person contacts?,No,46GP8,46GP8,none
3,46GP8-54,"In the video, did they touch a broom?",No,46GP8,46GP8,none
4,46GP8-60,Was a blanket one of the things they were cont...,No,46GP8,46GP8,none
...,...,...,...,...,...,...
1042306,LSKA2-5391,Was the person walking through a doorway befor...,after,LSKA2,LSKA2,none
1042307,LSKA2-5394,Were they walking through a doorway before or ...,after,LSKA2,LSKA2,none
1042308,LSKA2-5395,Were they walking through a doorway before or ...,after,LSKA2,LSKA2,none
1042309,LSKA2-5393,Was the person walking through a doorway befor...,after,LSKA2,LSKA2,none


In [43]:
total.to_csv('../exports/%s/other_formats/Total-question-updated.csv' % d_path, index = None)

In [33]:
test[test['key'] == 'YSKX3-1']

,key,question,answer,vid_id,gif_name,description


In [29]:
mismatch

,index,question,answer,key,vid_id,gif_name,description


In [23]:
templs = {}
for v_id in test_stsgs:
    with open('../exports/dataset_fixed/balanced/test/%s.txt' % v_id, 'rb') as f:
        QA = json.load(f)
        
    for q_id in QA:
        inside = QA[q_id]['id']
        if inside != q_id:
            t = QA[q_id]['attributes']['type']
            if t not in templs:
                templs[t] = 0
            templs[t] += 1 

In [24]:
templs

{'andObjRelExists': 13877,
 'actTime': 39395,
 'xorObjRelExists': 17043,
 'objWhat': 86612,
 'relExists': 2761,
 'actExists': 861,
 'objFirst': 5428,
 'objFirstVerify': 3320,
 'objLast': 8225,
 'objExists': 1483,
 'objWhatChoose': 9546,
 'objRelExists': 3401,
 'objFirstChoose': 3626,
 'objLastChoose': 3433,
 'objLastVerify': 3005,
 'objWhatGeneral': 501}